# Covid-19 Daily Tracking


In [1]:
import sys
import requests
import json
from datetime import datetime 
import pandas as pd

COUNTRY_LOC = 'malaysia'
API_URL = 'https://api.covid19api.com/total/country/' + COUNTRY_LOC

start_date = '2021-02-01'
to_date = datetime.today()
url_param = {'from': start_date, 'to': str(to_date)[:10]}

try:
    api_request = requests.get(API_URL, params=url_param, timeout=30)
except requests.exceptions.RequestException as e:
    print('Server Request Failed :', e)
    print('Exiting ....')
    sys.exit(0)

"""
if not api_request.ok:
    print('Server Request Failed :', api_request.status_code, api_request.reason)
    print('Exiting ....')
    sys.exit(0)
"""

df = pd.DataFrame(api_request.json())
df.set_index('Date', inplace=True)

In [2]:
df

,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active
Date,,,,,,,,,,,
2020-01-22T00:00:00Z,Malaysia,,,,,0,0,0,0,0,0
2020-01-23T00:00:00Z,Malaysia,,,,,0,0,0,0,0,0
2020-01-24T00:00:00Z,Malaysia,,,,,0,0,0,0,0,0
2020-01-25T00:00:00Z,Malaysia,,,,,0,0,3,0,0,3
2020-01-26T00:00:00Z,Malaysia,,,,,0,0,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...
2020-07-30T00:00:00Z,Malaysia,,,,,0,0,8964,124,8617,223
2020-07-31T00:00:00Z,Malaysia,,,,,0,0,8976,125,8644,207
2020-08-01T00:00:00Z,Malaysia,,,,,0,0,8985,125,8647,213


In [3]:
# remove unwanted Columns
df.drop(columns=['Country', 'CountryCode', 'Province', 'City', 'CityCode', 'Lat', "Lon"], inplace=True)

In [4]:
# rename columns date to strip of time info
for item in df.index:
    df.rename(index={item: str(item)[:10]}, inplace=True)
df

,Confirmed,Deaths,Recovered,Active
Date,,,,
2020-01-22,0,0,0,0
2020-01-23,0,0,0,0
2020-01-24,0,0,0,0
2020-01-25,3,0,0,3
2020-01-26,4,0,0,4
...,...,...,...,...
2020-07-30,8964,124,8617,223
2020-07-31,8976,125,8644,207
2020-08-01,8985,125,8647,213


In [5]:
# Population by Country data pulled from UN [source: https://population.un.org/wpp/Download/Standard/Population/] edited to conform to country's name
pop_df = pd.read_csv('../data/population.csv')

for index, item in enumerate(pop_df['Pop.(\'000)']):
    # strip empty spaces and convert to integer (in millions)
    pop_df.loc[index, 'Pop.(\'000)'] = int(item.replace(' ', ''))
    pop_df.loc[index, 'Pop.(\'000)'] = pop_df.loc[index, 'Pop.(\'000)'] * 1000 

    # rename Country's name to conform to covid's dataframe
    pop_df.loc[index, 'Country'] = pop_df.loc[index, 'Country'].lower().replace(' ', '-')

pop_df.set_index('Country', inplace=True)

## Data Crunching to form new dataset


In [6]:
daily_cases = []
daily_deaths = []
daily_recovered = []
mortality_rate = []
recovered_rate = []
active_rate = []
cases_per_million_pop = []

def append_rate_data(index):
    """ to append into list after calculation of
            - mortality rate
            - recovered rate
            - active rate
            - cases per 1mil population
    """
    
    # TODO: catch error for division with 0.0 (Not Working)
  
    mortality_rate.append(round((df.loc[df.index[index], 'Deaths'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    recovered_rate.append(round((df.loc[df.index[index], 'Recovered'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    active_rate.append(round((df.loc[df.index[index], 'Active'] / df.loc[df.index[index], 'Confirmed']) * 100, 2))
    cases_per_million_pop.append(round((df.loc[df.index[index], 'Confirmed'] / pop_df.loc[COUNTRY_LOC, 'Pop.(\'000)']) * 1000000, 0))    

      
for index, item in enumerate(df.index):
    if item == df.index[0]:
        new_case, new_death, new_recovered = df.loc[df.index[0], ['Confirmed', 'Deaths', 'Recovered']] 
        daily_cases.append(new_case)
        daily_deaths.append(new_death)
        daily_recovered.append(new_recovered)

    else:
        new_case, new_death, new_recovered = df.loc[df.index[index], ['Confirmed', 'Deaths', 'Recovered']] \
                                            - df.loc[df.index[index - 1], ['Confirmed', 'Deaths', 'Recovered']]
        daily_cases.append(new_case)
        daily_deaths.append(new_death)
        daily_recovered.append(new_recovered)

    df.loc[item, ['Daily Cases', 'Daily Deaths', 'Daily Recovered']] \
                  = daily_cases[index], daily_deaths[index], daily_recovered[index]

# release unwanted objects from memory
del daily_cases, daily_deaths, daily_recovered


df['Mortality Rates'] = round(df['Deaths'].divide(df ['Confirmed']).fillna(0.0), 2)
df['Recovered Rates'] = round(df['Recovered'].divide(df ['Confirmed']).fillna(0.0), 2)
df['Active Rates'] = round(df['Active'].divide(df ['Confirmed']).fillna(0.0), 2)
df['Cases per 1mil pop'] = round(df['Confirmed'] / pop_df.loc[COUNTRY_LOC, 'Pop.(\'000)'] * 1000000, 2)   

# TODO: add to df column

df


,Confirmed,Deaths,Recovered,Active,Daily Cases,Daily Deaths,Daily Recovered,Mortality Rates,Recovered Rates,Active Rates,Cases per 1mil pop
Date,,,,,,,,,,,
2020-01-22,0,0,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
2020-01-23,0,0,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
2020-01-24,0,0,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
2020-01-25,3,0,0,3,3.0,0.0,0.0,0.00,0.00,1.00,0.09
2020-01-26,4,0,0,4,1.0,0.0,0.0,0.00,0.00,1.00,0.12
...,...,...,...,...,...,...,...,...,...,...,...
2020-07-30,8964,124,8617,223,8.0,0.0,5.0,0.01,0.96,0.02,276.96
2020-07-31,8976,125,8644,207,12.0,1.0,27.0,0.01,0.96,0.02,277.33
2020-08-01,8985,125,8647,213,9.0,0.0,3.0,0.01,0.96,0.02,277.61


In [7]:
# replace NaN to zero
#df.fillna(0.0, inplace=True)
df.tail(7)

,Confirmed,Deaths,Recovered,Active,Daily Cases,Daily Deaths,Daily Recovered,Mortality Rates,Recovered Rates,Active Rates,Cases per 1mil pop
Date,,,,,,,,,,,
2020-07-28,8943,124,8607,212,39.0,0.0,6.0,0.01,0.96,0.02,276.31
2020-07-29,8956,124,8612,220,13.0,0.0,5.0,0.01,0.96,0.02,276.71
2020-07-30,8964,124,8617,223,8.0,0.0,5.0,0.01,0.96,0.02,276.96
2020-07-31,8976,125,8644,207,12.0,1.0,27.0,0.01,0.96,0.02,277.33
2020-08-01,8985,125,8647,213,9.0,0.0,3.0,0.01,0.96,0.02,277.61
2020-08-02,8999,125,8664,210,14.0,0.0,17.0,0.01,0.96,0.02,278.04
2020-08-03,9001,125,8668,208,2.0,0.0,4.0,0.01,0.96,0.02,278.10


In [8]:
# Max Daily Record
round(df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']].max(), 2)

Daily Cases        277.0
Daily Deaths         8.0
Daily Recovered    333.0
dtype: float64

In [9]:
round(df[['Daily Cases', 'Daily Deaths', 'Daily Recovered']].mean(), 2)

Daily Cases        46.16
Daily Deaths        0.64
Daily Recovered    44.45
dtype: float64

In [10]:
last_entry_date = df.index[len(df.index) - 1]
df.loc[last_entry_date]

Confirmed             9001.00
Deaths                 125.00
Recovered             8668.00
Active                 208.00
Daily Cases              2.00
Daily Deaths             0.00
Daily Recovered          4.00
Mortality Rates          0.01
Recovered Rates          0.96
Active Rates             0.02
Cases per 1mil pop     278.10
Name: 2020-08-03, dtype: float64